In [1]:
pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00


In [1]:
pip install skl2onnx

In [2]:
import time
import os
import pickle
import joblib
import tensorflow as tf
import onnx
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as ort

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Function to measure performance
def measure_performance(model, X_test):
    results = {}

    # --- 1. Pickle ---
    start_time = time.time()
    with open('model_pickle.pkl', 'wb') as f:
        pickle.dump(model, f)
    pickle_save_time = time.time() - start_time

    start_time = time.time()
    with open('model_pickle.pkl', 'rb') as f:
        loaded_model_pickle = pickle.load(f)
    pickle_load_time = time.time() - start_time

    pickle_file_size = os.path.getsize('model_pickle.pkl')

    start_time = time.time()
    pickle_pred = loaded_model_pickle.predict(X_test)
    pickle_predict_time = time.time() - start_time

    # --- 2. Joblib ---
    start_time = time.time()
    joblib.dump(model, 'model_joblib.pkl')
    joblib_save_time = time.time() - start_time

    start_time = time.time()
    loaded_model_joblib = joblib.load('model_joblib.pkl')
    joblib_load_time = time.time() - start_time

    joblib_file_size = os.path.getsize('model_joblib.pkl')

    start_time = time.time()
    joblib_pred = loaded_model_joblib.predict(X_test)
    joblib_predict_time = time.time() - start_time

    # --- 3. ONNX ---
    onnx_model = convert_sklearn(model, initial_types=[('input', FloatTensorType([None, X_train.shape[1]]))])
    onnx.save_model(onnx_model, 'model.onnx')

    onnx_save_time = time.time() - start_time

    onnx_file_size = os.path.getsize('model.onnx')

    start_time = time.time()
    onnx_model_loaded = onnx.load('model.onnx')
    onnx_load_time = time.time() - start_time

    onnx_session = ort.InferenceSession('model.onnx')
    start_time = time.time()
    onnx_pred = onnx_session.run(None, {'input': X_test.astype(np.float32)})[0]
    onnx_predict_time = time.time() - start_time

    # --- 4. TensorFlow ---
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Dense(3, activation='softmax', input_dim=X_train.shape[1])
    ])
    keras_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    keras_model.fit(X_train, y_train, epochs=10, verbose=0)

    start_time = time.time()
    keras_model.save('model_tensorflow.h5')
    tensorflow_save_time = time.time() - start_time

    tensorflow_file_size = os.path.getsize('model_tensorflow.h5')

    start_time = time.time()
    loaded_model_tf = tf.keras.models.load_model('model_tensorflow.h5')
    tensorflow_load_time = time.time() - start_time

    start_time = time.time()
    tensorflow_pred = loaded_model_tf.predict(X_test)
    tensorflow_predict_time = time.time() - start_time

    # Results
    results = {
        'pickle': {
            'save_time': pickle_save_time,
            'load_time': pickle_load_time,
            'size': pickle_file_size,
            'pred_time': pickle_predict_time
        },
        'joblib': {
            'save_time': joblib_save_time,
            'load_time': joblib_load_time,
            'size': joblib_file_size,
            'pred_time': joblib_predict_time
        },
        'onnx': {
            'save_time': onnx_save_time,
            'load_time': onnx_load_time,
            'size': onnx_file_size,
            'pred_time': onnx_predict_time
        },
        'tensorflow': {
            'save_time': tensorflow_save_time,
            'load_time': tensorflow_load_time,
            'size': tensorflow_file_size,
            'pred_time': tensorflow_predict_time
        }
    }

    return results

# Run the experiment
results = measure_performance(model, X_test)

# Display the results
for method, metrics in results.items():
    print(f"Method: {method}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.5f}")
    print()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Method: pickle
  save_time: 0.00037
  load_time: 0.00010
  size: 834.00000
  pred_time: 0.00050

Method: joblib
  save_time: 0.00130
  load_time: 0.00067
  size: 991.00000
  pred_time: 0.00025

Method: onnx
  save_time: 0.01313
  load_time: 0.00018
  size: 686.00000
  pred_time: 0.00032

Method: tensorflow
  save_time: 0.02091
  load_time: 0.02939
  size: 17960.00000
  pred_time: 0.14243

